In [0]:
FIT_GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/rbf/bosz/bosz_5000_full_2/fit'
RBF_GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/rbf/bosz/bosz_5000_full_2/fit-rbf-2'

In [0]:
%matplotlib inline

In [0]:
import sys

# Allow load project as module
sys.path.insert(0, '../../../..')

In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

In [0]:
import matplotlib     
matplotlib.rcParams.update({'font.size': 14})

# Load stellar model grid

In [0]:
from pfsspec.core.grid import ArrayGrid, RbfGrid
from pfsspec.stellar.grid import ModelGrid
from pfsspec.stellar.grid.bosz import Bosz
from pfsspec.stellar.grid.phoenix import Phoenix

## Grid of fitted parameters

In [0]:
fn = os.path.join(FIT_GRID_PATH, 'spectra.h5')
fit_grid = ModelGrid(Bosz(normalized=True), ArrayGrid)
fit_grid.preload_arrays = True
fit_grid.load(fn, format='h5')

In [0]:
fit_grid.wave

In [0]:
for k in fit_grid.grid.axes.keys():
    print(k, fit_grid.grid.axes[k].values)

## Rbf grid

In [0]:
fn = os.path.join(RBF_GRID_PATH, 'spectra.h5')
rbf_grid = ModelGrid(Bosz(normalized=True), RbfGrid)
rbf_grid.preload_arrays = False
rbf_grid.load(fn, format='h5')

In [0]:
rbf_grid.wave

In [0]:
for k in rbf_grid.grid.axes.keys():
    print(k, rbf_grid.grid.axes[k].values)

In [0]:
rbf_grid.grid.values.keys()

# Plot

In [0]:
M_H = 0
T_eff = 4500
log_g = 1
a_M = 0
C_M = 0

rbf_idx = rbf_grid.grid.get_nearest_index(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M)
rbf_idx

In [0]:
array_idx = fit_grid.grid.get_nearest_index(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M)
array_idx

In [0]:
def plot_params(rbf_grid, array_grid, rbf, array, idx=2, param_idx=0):
    axes = list(rbf_grid.axes.values())
    pp = axes[idx].values.size
    rr = int(pp / 4 + 0.5)
    f, axs = plt.subplots(rr, 4, figsize=(16, 4 * rr))
    for p in range(pp):
        i = p // 4
        j = p % 4
        
        coords = [np.arange(axes[k].values.size) for k in range(3)]
        del coords[idx]
        coords = np.meshgrid(*coords)
        coords.insert(idx, np.full_like(coords[0], p))
        rbf_values = rbf(*[ x.flatten() for x in coords])
        rbf_values = rbf_values[..., param_idx].reshape(coords[0].shape).T
        
        s = (array.ndim - 1) * [slice(None)]
        s[idx] = p
        s = tuple(s)
        array_values = array[s][..., param_idx]
        
        image = array_values - rbf_values
        
        l = axs[i, j].imshow(image, aspect='auto')
        f.colorbar(l, ax=axs[i, j])
        axs[i, j].set_xlabel('param: {} | slice: {}'.format(param_idx, p))

In [0]:
rbf = rbf_grid.rbf_grid.values['blended_0']
array = fit_grid.array_grid.values['blended_0'][:, :, :, array_idx[-2], array_idx[-1]]
for pi in range(rbf.nodes.shape[-1]):
    plot_params(rbf_grid.rbf_grid, fit_grid.array_grid, rbf, array, param_idx=pi)